# <font color=blue><div align="center">Seats Allocation within an Aircraft</div></font>

### <font color=blue><div align="center">29-02-2024</div></font>

## Modules

In [2]:
# Basic modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Gurobi module
from gurobipy import *

## Data Extraction

In [18]:
number_of_seats = 174
weight_f = 70
weight_m = 85
weight_h = 92.5
number_of_rows = 29
number_of_columns = 6

In [4]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('./DataSeating 2024.xlsx', sheet_name=0 , skipfooter=2)

In [5]:
# Display the DataFrame
df

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,NaN,1.0,NaN,01:15:00
1,2,1.0,NaN,NaN,00:00:00
2,3,NaN,1.0,NaN,00:00:00
3,4,1.0,NaN,NaN,00:00:00
4,5,NaN,2.0,NaN,05:25:00
...,...,...,...,...,...
106,107,1.0,NaN,NaN,03:30:00
107,108,2.0,NaN,NaN,05:25:00
108,109,2.0,NaN,NaN,05:25:00
109,110,2.0,NaN,NaN,01:35:00


In [17]:

Passagers = dict()
i = 1

for group in df.itertuples():
    for j in range(2,5):
        if not pd.isna(group[j]):
            for k in range(int(group[j])):
                Passagers[i] = {'gender': j-2, 'group':group[1], 'transit':group[5].hour*60 + group[5].minute }
                i+=1    
    
Passagers

{1: {'gender': 1, 'group': 1, 'transit': 75},
 2: {'gender': 0, 'group': 2, 'transit': 0},
 3: {'gender': 1, 'group': 3, 'transit': 0},
 4: {'gender': 0, 'group': 4, 'transit': 0},
 5: {'gender': 1, 'group': 5, 'transit': 325},
 6: {'gender': 1, 'group': 5, 'transit': 325},
 7: {'gender': 0, 'group': 6, 'transit': 0},
 8: {'gender': 0, 'group': 7, 'transit': 190},
 9: {'gender': 0, 'group': 8, 'transit': 80},
 10: {'gender': 1, 'group': 8, 'transit': 80},
 11: {'gender': 1, 'group': 9, 'transit': 140},
 12: {'gender': 0, 'group': 10, 'transit': 0},
 13: {'gender': 0, 'group': 11, 'transit': 235},
 14: {'gender': 1, 'group': 11, 'transit': 235},
 15: {'gender': 1, 'group': 12, 'transit': 70},
 16: {'gender': 1, 'group': 13, 'transit': 0},
 17: {'gender': 0, 'group': 14, 'transit': 0},
 18: {'gender': 1, 'group': 14, 'transit': 0},
 19: {'gender': 1, 'group': 15, 'transit': 360},
 20: {'gender': 1, 'group': 16, 'transit': 95},
 21: {'gender': 0, 'group': 17, 'transit': 60},
 22: {'gender

## Model

Defining the relevant variables of the problem

In [ ]:

m = Model("Seats_Allocation")

AssignmenVarDict = {(i, j, k) : m.addVar(vtype = GRB.BINARY, name=f'passager_{i}_{j}_{k}') for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns + 1) for k in range(len())}